參考
https://www.kaggle.com/apapiu/regularized-linear-models

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
testIds = test['Id']

In [3]:
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))


In [4]:
from scipy.special import boxcox1p
#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
#skewed_feats = skewed_feats[abs(skewed_feats) > 0.75]
print(skewed_feats.shape)
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
#all_data[skewed_feats] = boxcox1p(all_data[skewed_feats], 0.15)  
 

(21,)


In [5]:
all_data = pd.get_dummies(all_data)

In [6]:
#filling NA's with the mean of the column:
all_data = all_data.fillna(all_data.mean())
#all_data = all_data.fillna(all_data.median())

In [7]:
#creating matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]

#log transform the target:
y = np.log1p(train.SalePrice)



In [8]:
print(X_train.shape,X_test.shape,y.shape)

(1460, 288) (1459, 288) (1460,)


In [9]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.110874,4.189655,9.042040,7,5,2003,2003,5.283204,6.561031,0.0,...,0,0,0,1,0,0,0,0,1,0
1,3.044522,4.394449,9.169623,6,8,1976,1976,0.000000,6.886532,0.0,...,0,0,0,1,0,0,0,0,1,0
2,4.110874,4.234107,9.328212,7,5,2001,2002,5.093750,6.188264,0.0,...,0,0,0,1,0,0,0,0,1,0
3,4.262680,4.110874,9.164401,7,5,1915,1970,0.000000,5.379897,0.0,...,0,0,0,1,1,0,0,0,0,0
4,4.110874,4.442651,9.565284,8,5,2000,2000,5.860786,6.486161,0.0,...,0,0,0,1,0,0,0,0,1,0


In [10]:
X_test.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,3.044522,4.394449,9.360741,5,6,1961,1961,0.000000,6.150603,4.976734,...,0,0,0,1,0,0,0,0,1,0
1,3.044522,4.406719,9.565775,6,6,1958,1958,4.691348,6.828712,0.000000,...,0,0,0,1,0,0,0,0,1,0
2,4.110874,4.317488,9.534668,5,5,1997,1998,0.000000,6.674561,0.000000,...,0,0,0,1,0,0,0,0,1,0
3,4.110874,4.369448,9.208238,6,6,1998,1998,3.044522,6.401917,0.000000,...,0,0,0,1,0,0,0,0,1,0
4,4.795791,3.784190,8.518392,8,5,1992,1992,0.000000,5.575949,0.000000,...,0,0,0,1,0,0,0,0,1,0


In [16]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split


#model = linear_model.Lasso(alpha = 0.005).fit(X_train,y)
#rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
#print(rmse, rmse.mean())

model = LassoCV(alphas = [1, 0.1, 0.001, 0.0005, 0.0007]).fit(X_train, y)
#kf = KFold(5, shuffle=True, random_state=42).get_n_splits(train.values)
rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
print(rmse, rmse.mean())

coef = pd.Series(model.coef_, index = X_train.columns)
print('best alpha=',model.alpha_,"Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")



[ 0.10416707  0.13196908  0.12552436  0.10667935  0.14739251] 0.123146473975
best alpha= 0.0005 Lasso picked 110 variables and eliminated the other 178 variables


In [17]:
#測試用GridSearchCV驗證LassoCV
from sklearn.model_selection import GridSearchCV
parameters= {'alpha':[1, 0.1, 0.001, 0.0005, 0.0007]}
modelL = linear_model.Lasso()

bestModelL = GridSearchCV(modelL, parameters, cv=5)
bestModelL.fit(X_train,y)
rmse= np.sqrt(-cross_val_score(bestModelL, X_train, y, scoring="neg_mean_squared_error", cv = 5))
print(rmse, rmse.mean())

[ 0.10330973  0.13147319  0.12552436  0.10667935  0.14739251] 0.122875826753


In [18]:
#測試mean_squared_error
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_train)
print('mean_squared_error ',np.sqrt(mean_squared_error(y, y_pred)))


mean_squared_error  0.105025782621


In [19]:
coef.shape

(288,)

In [20]:
y_pred = model.predict(X_test)

out = pd.DataFrame()
out['Id'] = testIds
out['SalePrice'] =  np.expm1(y_pred)
print(out.shape)
out.to_csv('submit.csv',index=False)

(1459, 2)
